###Evaluation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
path = "/content/gdrive/My Drive/"

In [0]:
from __future__ import absolute_import, division, print_function

#import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
#import keras

###Silence Warnings
- 0: all messages are logged (default behavior)
- 1: INFO messages are not printed
- 2: INFO and WARNING messages are not printed
- 3: INFO, WARNING, and ERROR messages are not printed

In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

###Model

In [0]:
def deficit_cnn_model(features, labels, mode):

    ''' Input Layer '''
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.reshape(features, [-1, 50, 5, 1])

    ''' Convolutional Layer #1 '''
    # Computes 32 features using a 5x5 filter with ReLU activation.
    #   Padding is added to preserve width and height.
    # Tensor Shape: [batch_size, 50, 5, 32]
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu
    )

    ''' Pooling Layer #1 '''
    # Max pooling layer with a 2x2 filter and stride of 2
    # Tensor Shape: [batch_size, 25, 2, 32]
    pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = [2, 2],
        strides = 2
    )

    ''' Convolutional Layer #2 '''
    # Computes 64 features using a 5x5 filter.
    #   Padding is added to preserve width and height.
    # Tensor Shape: [batch_size, 25, 2, 64]
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu
    )

    ''' Pooling Layer #2 '''
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Tensor Shape: [batch_size, 12, 1, 64]
    pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = [2, 2],
        strides = 2
    )

    ''' Flatten tensor into a batch of vectors '''
    # Output Tensor Shape: [batch_size, 768]
    pool2_flat = tf.reshape(pool2, [-1, 12 * 64])

    ''' Dense Layer '''
    # Densely connected layer with 1024 neurons
    # Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(
        inputs = pool2_flat,
        units = 1024,
        activation = tf.nn.relu
    )
    
    ''' Add dropout operation '''
    # 0.6 probability that element will be kept
    # Tensor Shape: [batch_size, 1024]
    dropout = tf.layers.dropout(
        inputs = dense,
        rate = 0.4,
        training = mode == tf.estimator.ModeKeys.TRAIN
    )


    ''' Logits layer '''
    # Input Tensor Shape: [batch_size, 1024]
    # Tensor Shape: [batch_size, 1024]
    logits = tf.layers.dense(
        inputs = dropout,
        units = 10
    )

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),

        # Add `softmax_tensor` to the graph.
        # It is used for PREDICT and by the `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:

        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step()
        )

        return tf.estimator.EstimatorSpec(
            mode = mode,
            loss = loss,
            train_op = train_op
        )

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels,
            predictions = predictions["classes"]
        )
    }

    return tf.estimator.EstimatorSpec(
        mode = mode,
        loss = loss,
        eval_metric_ops = eval_metric_ops
    )

##Load Data and Setup Logging

###Data

In [0]:
# Shape: [606, 250]
train_data = np.loadtxt(path + "training_set_80.txt")
train_labels = np.loadtxt(path + "training_labels_80.txt", dtype = np.int32)

eval_data = np.loadtxt(path + "training_set_80.txt")
eval_labels = np.loadtxt(path + "training_labels_80.txt", dtype = np.int32)

###Logging

In [0]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors = tensors_to_log,
    every_n_iter = 50
)

##Create Estimator and Train

###Create Estimator

In [8]:
classifier = tf.estimator.Estimator(model_fn = deficit_cnn_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzbfgg33t', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f591156e6d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


###Training

In [0]:
# Shape: [606, 50, 5, 1]
input_layer = tf.reshape(train_data, [-1, 50, 5, 1])

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = train_data,
    y = train_labels,
    batch_size = 10,
    num_epochs = None,
    shuffle = True
)

In [10]:
classifier.train(
    input_fn = train_input_fn,
    steps = 1000,
    #hooks = [logging_hook]
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoin

##Evaluation

In [11]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = eval_data,
    y = eval_labels,
    num_epochs = 1,
    shuffle = False
)
eval_results = classifier.evaluate(input_fn = eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-31T03:05:35Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpzbfgg33t/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-31-03:05:35
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.5247525, global_step = 1000, loss = 0.7020097
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpzbfgg33t/model.ckpt-1000
{'accuracy': 0.5247525, 'loss': 0.7020097, 'global_step': 1000}
